In [1]:
import numpy as np
import pandas as pd
import random


def age_series(age=30, freq=12, yrs=40, **kwargs):
    age_array = np.ones(freq * yrs + 1) / freq
    age_array[0] = age
    age_array = np.cumsum(age_array)
    age_s = pd.Series(age_array, name="age")
    return age_s


def salary_series(
    salary=100000, salary_growth=0.04, freq=12, yrs=40, age=30, ret_age=65, **kwargs
):
    income_period = ret_age - age
    salary_growth_per_period = (1 + salary_growth) ** (1 / freq) - 1
    salary_growth_array = np.ones(freq * yrs + 1) * (
        1 + salary_growth_per_period
    )
    salary_growth_array[0] = salary
    salary_array = np.cumprod(salary_growth_array)
    salary_array[(income_period * freq) :] = 0
    salary_s = pd.Series(salary_array, name="salary")
    return salary_s


def invest_series(
    salary_s=None,
    invest_pct=0.15,
    freq=12,
    ret_inc=100000,
    age=30,
    ret_age=65,
    **kwargs
):
    income_period = ret_age - age
    if salary_s is None:
        salary_s = salary_series()
    invest_pct_per_period = invest_pct / freq
    invest = salary_s * invest_pct_per_period
    invest[income_period * freq :] = -1 * ret_inc / freq
    invest.name = "invest"
    return invest

# def fund_growth(fund_value_start, s_model_results, b_model_results, stock_pct_strt = .80):
#     bond_pct_strt = 1 - stock_pct_strt

#     # create stock and bond index return arrays. 
#     stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
#     bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

#     stock_array = np.insert(stock_return, 0, fund_value_start * stock_pct_strt, axis=1)
#     bond_array = np.insert(bond_return, 0, fund_value_start * bond_pct_strt, axis=1)

#     # the last return value is not used so we add a 1 to the end to return the array to its original length
#     ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
#     stock_return = np.append(stock_return, ones_to_append, axis=1)
#     bond_return = np.append(bond_return, ones_to_append, axis=1)

def mortality_sim(age=32, ret_age=65, sex="F", smoker="NS", n_scen=1000, mi=0.99, **kwargs):
    mort_df = pd.read_csv("MortalityTables/cso2017.csv")
    mort_df = mort_df.set_index(["Sex", "SmokingStatus", "IssueAge"])
    row = mort_df.loc[(sex, smoker, ret_age)]
    arr = np.empty(n_scen)

    for n in range(n_scen):
        d = 0
        j = ret_age
        i = row[d] / 1000 * (mi ** (j - age))
        while random.random() > i:
            j += 1
            d += 1
            if d < len(row):  # Check if index is out of range
                i = row[d] / 1000 * (mi ** (j - age))
            else:
                break
        arr[n] = j

    df = pd.DataFrame(arr, columns=["age_at_death"])

    return df


In [4]:
salary = 200000
config = {
    'salary':salary,
    'salary_growth':0.04,
    'freq':12,
    'yrs':40,
    'age':30,
    'ret_age':65,
    'test':2
}
salary_s = salary_series(**config)
print(salary_s)

0      200000.000000
1      200654.747956
2      201311.639387
3      201970.681310
4      202631.880764
           ...      
476         0.000000
477         0.000000
478         0.000000
479         0.000000
480         0.000000
Name: salary, Length: 481, dtype: float64


In [12]:
from allocation import allocation

allocation_df = allocation()
age_s = age_series()
invest_s = invest_series()
salary_s = salary_series()

df = pd.concat([age_s, allocation_df, invest_s, salary_s], axis=1)
print(df)


           age  equity  bond       invest         salary
0    30.000000     0.9   0.1  1250.000000  100000.000000
1    30.083333     0.9   0.1  1254.092175  100327.373978
2    30.166667     0.9   0.1  1258.197746  100655.819694
3    30.250000     0.9   0.1  1262.316758  100985.340655
4    30.333333     0.9   0.1  1266.449255  101315.940382
..         ...     ...   ...          ...            ...
476  69.666667     0.4   0.6 -8333.333333       0.000000
477  69.750000     0.4   0.6 -8333.333333       0.000000
478  69.833333     0.4   0.6 -8333.333333       0.000000
479  69.916667     0.4   0.6 -8333.333333       0.000000
480  70.000000     0.4   0.6 -8333.333333       0.000000

[481 rows x 5 columns]
